In [2]:
#!pip install firebase_admin

In [4]:
import firebase_admin
from firebase_admin import credentials
from firebase_admin import firestore
serviceAccountKey_json_file = "firestore.json"
cred = credentials.Certificate(serviceAccountKey_json_file)
firebase_admin.initialize_app(cred)

db = firestore.client()

In [5]:
# creating a new collection and initializing it with data fields
db.collection("persons").add(
    {
        "name": "John", 
        "age": 48,
        "married": True
    }
)

(DatetimeWithNanoseconds(2021, 4, 29, 23, 47, 3, 558650, tzinfo=datetime.timezone.utc),
 <google.cloud.firestore_v1.document.DocumentReference at 0x18946605b20>)

# CREATING DATA TO THE CLOUD FIRESTORE

In [12]:
# auto ids allow us to create duplicate data over an over if we re-run it

In [6]:
# Add documents with auto IDs
data = dict(name="John", age=40, employed=True)
db.collection("people").add(data)

(DatetimeWithNanoseconds(2021, 4, 29, 23, 55, 23, 48408, tzinfo=datetime.timezone.utc),
 <google.cloud.firestore_v1.document.DocumentReference at 0x189465b87c0>)

In [13]:
# Set documents with auto IDs
data = dict(name="Jane David", age=32, employed=False)
db.collection("people").document().set(data) # doc_name is not passed here.. so auto IDs

update_time {
  seconds: 1619741567
  nanos: 879226000
}

In [15]:
# Set documents with known IDs
data = dict(name="Jane Doe", age=40, employed=False)
id_no = 1  # use for loop in seriious projects
doc_name = str(id_no).zfill(4) + "_" + (data['name']).lower().replace(" ", "_") # just setting doc_name like 0001_name_etc
db.collection("people").document(doc_name).set(data)  # adding document to collection with document reference

update_time {
  seconds: 1619741941
  nanos: 257674000
}

In [16]:
# Merging data or update sometimes
data = dict(address="#566, 24th Avenue, Ist Main, Bengalore", age=60) # earlier age of Jane Doe was 40 now changeing it 60
db.collection("people").document(doc_name).set(data, merge=True) 
# using same doc_name from Jane Doe section of set doc with known ids

update_time {
  seconds: 1619742027
  nanos: 390973000
}

In [17]:
# adding movies which Jane Doe watched |  ==== Collection inside a document
# use add for autoIds or set for our own ids for movies, here i use unique ids
data = dict(name="Avengers")
db.collection("people").document(doc_name).collection("movies").document("001_Avengers").set(data)

data = dict(name="Harry Potter")
db.collection("people").document(doc_name).collection("movies").document("002_Harry_Potter").set(data)

update_time {
  seconds: 1619742700
  nanos: 171814000
}

# Read Data for FireStore

In [18]:
# getting a document with a known ID (not generated Ids)
result = db.collection("people").document(doc_name).get()
if result.exists:
    print(result.to_dict())

{'age': 60, 'name': 'Jane Doe', 'address': '#566, 24th Avenue, Ist Main, Bengalore', 'employed': False}


In [20]:
# getting all documents in a collection
docs = db.collection("people").get()
for doc in docs:
    print(doc.to_dict())

{'name': 'Jane Doe', 'address': '#566, 24th Avenue, Ist Main, Bengalore', 'age': 60, 'employed': False}
{'employed': False, 'name': 'Jane Doe', 'age': 40}
{'employed': True, 'name': 'John', 'age': 40}
{'employed': False, 'name': 'Jane David', 'age': 32}


In [ ]:
# QUERYING - where, 

In [24]:
# equal - work with integer, float 
docs = db.collection("people").where("age", "==", 40).get() # where
for doc in docs:
    print(doc.to_dict())

{'age': 40, 'employed': False, 'name': 'Jane Doe'}
{'name': 'John', 'age': 40, 'employed': True}


In [22]:
# greater than - work with only integers 
docs = db.collection("people").where("age", ">", 40).get() # where
for doc in docs:
    print(doc.to_dict())

{'employed': False, 'address': '#566, 24th Avenue, Ist Main, Bengalore', 'name': 'Jane Doe', 'age': 60}


In [25]:
# same way use == != > < >= <=

In [26]:
# equal - work with strings 
docs = db.collection("people").where("name", "==", "John").get() # where
for doc in docs:
    print(doc.to_dict())

{'age': 40, 'name': 'John', 'employed': True}


In [33]:
# array search
docs = db.collection("people").where("socials", "array_contains", "YouTube").get() # where
for doc in docs:
    print(doc.to_dict())

{'name': 'Jane Doe', 'employed': False, 'socials': ['Facebook', 'YouTube'], 'address': '#566, 24th Avenue, Ist Main, Bengalore', 'age': 60}


In [35]:
# IN OPERATOR
docs = db.collection("people").where(
    "address", "in", ["#566, 24th Avenue, Ist Main, Bengalore", "Africa"] # this shoudl be an array for IN to work
).get() # where in
for doc in docs:
    print(doc.to_dict())

{'socials': ['Facebook', 'YouTube'], 'address': '#566, 24th Avenue, Ist Main, Bengalore', 'age': 60, 'employed': False, 'name': 'Jane Doe'}
